In [1]:
import numpy as np
from numba import njit
import pickle

from scipy.spatial.distance import cdist, pdist

In [2]:
var = pickle.load(open('var.pickle', 'rb'))


In [3]:
BOX_miu = var['BOX_miu']
BOXG = var['BOXG']
NB = var['NB']
interval1 = var['interval1']
interval2 = var['interval2']

In [4]:
print("----- BOX_miu -----")
print(BOX_miu)

print("----- BOXG -----")
print(BOXG)

print("----- NB -----")
print(NB)

print("----- interval1 -----")
print(interval1)

print("----- interval2 -----")
print(interval2)

----- BOX_miu -----
[[14.95026657 -2.49513858  1.08329909 ... -0.77966473  1.33400379
   0.14598842]
 [12.91330806 -4.78882708  3.26570517 ...  0.97170965  2.42642645
  -0.18516359]
 [ 9.29097232  6.39343343  2.50659111 ...  5.98935651 -0.29082587
   2.321144  ]
 ...
 [ 1.40713921  3.8946079  -2.62881626 ...  0.60813352  0.69324394
   1.41036463]
 [ 5.73160474 -0.730143    1.43264209 ... -3.22497985  0.25149412
  -0.87659361]
 [-1.3687837   0.88025944 -2.2674405  ...  0.34591206 -0.71409173
  -0.40471209]]
----- BOXG -----
[0.379728   0.2945503  0.26140656 0.25803701 0.17646112 0.17002029
 0.16761585 0.15754701 0.15276545 0.1495008  0.14147018 0.14203024
 0.13794973 0.13381941 0.12976737 0.10721593 0.10360611 0.10260562
 0.09731847 0.09582816 0.09488369 0.09124841 0.08532286 0.08463549
 0.08500186 0.08261603 0.0817933  0.08182261 0.08201287 0.15526337
 0.07960983 0.07832987 0.07622987 0.07686049 0.07406293 0.07482985
 0.07405985 0.07423108 0.07351631 0.07272718 0.07352328 0.07066242
 0

In [5]:
def ChessBoard_online_projection_std(BOX_miu,BOXMT,NB,interval1,interval2):
    Centers = []
    ModeNumber = 0
    n = 2
    
    for ii in range(NB):
        Reference = BOX_miu[ii]
        distance1 = np.zeros((NB,1))
        distance2 = np.zeros((NB,1))
        for i in range(NB):
            distance1[i] = pdist([list(Reference), list(BOX_miu[i])], 'euclidean')

            distance2[i] = np.sqrt(pdist([list(Reference), list(BOX_miu[i])], 'cosine'))
            

        Chessblocak_typicality = []
        for i in range(NB):
            if distance1[i]<n*interval1 and distance2[i]<n*interval2:
                Chessblocak_typicality.append(BOXMT[i])
        if max(Chessblocak_typicality) == BOXMT[ii]:
            Centers.append(Reference)
            ModeNumber += 1
    return Centers,ModeNumber

In [6]:
%%time
Centers_std, ModeNumber_std = ChessBoard_online_projection_std(BOX_miu,BOXG,NB,interval1,interval2)

Wall time: 6.98 s


In [ ]:
print("----- Centers -----")
print(Centers_std)

print("----- Mode Number -----")
print(ModeNumber_std)

In [7]:
@njit
def ChessBoard_online_projection_njit(BOX_miu,BOXMT,NB,interval1,interval2):
    Centers = []
    ModeNumber = 0
    n = 2
    
    for ii in range(NB):
        Reference = BOX_miu[ii]
        distance1 = np.zeros((NB,1))
        distance2 = np.zeros((NB,1))
        for i in range(NB):
            
            
            # pdist euclidean
            # distance1[i] = pdist(np.vstack((Reference, BOX_miu[i])), 'euclidean')
            W, = Reference.shape
            
            aux = 0
            for iii in range(W):
                aux += (Reference[iii] - BOX_miu[i, iii])**2 
                
            distance1[i] = aux**.5
                
            # pdist cosine
            # distance2[i] = np.sqrt(pdist(np.vstack((Reference, BOX_miu[i])), 'cosine'))
            
            
            num = 0
            den1 = 0
            den2 = 0
            for iii in range(W):
                num += Reference[iii]*BOX_miu[i, iii]
                den1 += Reference[iii]**2
                den2 += BOX_miu[i, iii]**2
            
            d2 = (1 - num/(den1**.5 * den2**.5) )
            if d2 < 0:
                distance2[i] = 0
            else:
                distance2[i] = d2**.5
              
        
        Chessblocak_typicality = []
        for i in range(NB):
            if distance1[i]<n*interval1 and distance2[i]<n*interval2:
                Chessblocak_typicality.append(BOXMT[i])
        if max(Chessblocak_typicality) == BOXMT[ii]:
            Centers.append(Reference)
            ModeNumber += 1
    return Centers,ModeNumber

In [10]:
%%time
Centers_njit, ModeNumber_njit = ChessBoard_online_projection_njit(BOX_miu,BOXG,NB,interval1,interval2)

Wall time: 35.9 ms


In [ ]:
print("----- Centers -----")
print(Centers_njit)

print("----- Mode Number -----")
print(ModeNumber_njit)

In [11]:
print("----- Centers -----")
print(sum([Centers_std[i] - Centers_njit[i] for i in range(len(Centers_std))]))
print("STD Type: ", type(Centers_std))
print("NJIT Type: ", type(Centers_njit))

print("----- Mode Number -----")
print(ModeNumber_std - ModeNumber_njit)
print("STD Type: ", type(ModeNumber_std))
print("NJIT Type: ", type(ModeNumber_njit))

----- Centers -----
[0. 0. 0. 0. 0. 0. 0. 0.]
STD Type:  <class 'list'>
NJIT Type:  <class 'list'>
----- Mode Number -----
0
STD Type:  <class 'int'>
NJIT Type:  <class 'int'>
